In [1]:
import pandas as pd
import numpy as np
import torch
from utils.nn_data_classifier import load_data, Classifier
from utils.preprocess import preprocess
from utils.nn_data_classifier import Classifier

In [2]:
# classi = Classifier(max_drop=-0.10, max_raise=0.10)
# historical_data = load_data()

# classified_data = classi.classify_data(historical_data)
# classified_data.to_csv('./historicals/BTC-min-01-01.csv')
# classified_data

In [3]:
classified_data_path = './historicals/BTC-min-01-01.csv'
data = pd.read_csv(classified_data_path, parse_dates=['Day']).iloc[:, 1:]
data.sort_values("Day", inplace=True)
data = data[data.Classification != 2]
data


,Day,Price,Volume,Classification
0,2017-01-01 00:01:00,966.34,7.610000,1.0
1,2017-01-01 00:02:00,966.37,8.087376,1.0
2,2017-01-01 00:03:00,966.37,0.000000,1.0
3,2017-01-01 00:04:00,966.37,0.000000,1.0
4,2017-01-01 00:05:00,966.43,0.107000,1.0
...,...,...,...,...
2674550,2022-02-28 15:12:00,39764.94,1.284295,1.0
2674551,2022-02-28 15:13:00,39774.64,9.949596,1.0
2674552,2022-02-28 15:14:00,39797.66,0.661230,1.0
2674553,2022-02-28 15:15:00,39848.52,2.110273,1.0


In [4]:
SEQ_LEN = 60

In [5]:
processor = preprocess(sequence_length=SEQ_LEN)
training, testing  = processor.preprocess(dataframe=data, validation_size=0.1)

/home/ayrton/Projects/Quant-Trader/utils/preprocess.py:161: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(sequential_data)


In [53]:
from torch import nn
from torch.nn.functional import relu
class RNN_module(nn.Module):
    def __init__(self, hidden_size, input_size, output_size, num_layers):
        super().__init__()
        self._num_layers = num_layers
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size

        self.lstm = nn.LSTM(input_size = self._input_size, hidden_size = self._hidden_size, 
                            num_layers = self._num_layers, batch_first = True)

        self.fc = nn.Linear(in_features=self._hidden_size, out_features= self._output_size)

    def forward(self, input):
        h_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size).double()
        c_0 = torch.zeros(self._num_layers, input.size(0), self._hidden_size).double()

        input = input.double()
        print(h_0.dtype)

        lstm_output, h_n, c_n = self.lstm(input, (h_0, c_0))

        logist = self.fc(lstm_output[:, -1, :])
        scores = relu(logist)
        return scores

In [54]:
from torch.utils.data import DataLoader
learning_rate = 0.01
dim_size = training._sequence.size(dim=-1)
hidden_size = 60
batch_size = 64
number_of_classes = training._labels.size(dim=-1)
train_dataloader = DataLoader(training, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(testing, batch_size = batch_size, shuffle = True)

model = RNN_module(hidden_size = hidden_size, input_size = dim_size,
                     output_size = number_of_classes, num_layers = 2)

In [8]:
labels = training._labels
classes = torch.unique(labels, dim = 0)
weights = torch.zeros((classes.shape[0],))
total = torch.tensor(len(training))

for idx, _class in enumerate(classes):
    weights = weights + _class * (1 - labels[labels == _class].sum() / total)

weights

tensor([0.5736, 0.4264], dtype=torch.float64)

In [9]:
criterion = nn.CrossEntropyLoss(weight= weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [38]:
def train_loop(train_dataloader: DataLoader, epochs: int):
    n_of_steps = len(train_dataloader) #total number of batches

    for epoch in range(epochs):
        for current_batch, (sequence, label) in enumerate(train_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            loss = criterion(input=outputs, target = label)

            #backward: lets update the parameters given the current loss
            optimizer.zero_grad() #nullifies the current gradients. If you don't do this, gradients will be added up (you don't want that)
            loss.backward() #computates the bwrd-prop gradient for each model parameter
            optimizer.step() #updates the model current parameter using the gradients.

            if (current_batch + 1) % 500 == 0:
                print(f"epoch {epoch+1}/{epochs}, current step(batch): {current_batch+1}/{n_of_steps}, loss = {loss.item():.4f} ")

In [13]:
def test_loop(test_dataloader: DataLoader):
    with torch.no_grad():
        n_corrects = 0
        n_samples = 0

        for current_batch, (sequence, label) in enumerate(train_dataloader):

            #forward: we are calculating the loss given the parameters
            outputs = model(sequence)
            predictions = (outputs == outputs.max(dim=1, keepdim=True)[0]).to(dtype=torch.int32) #one hot encode the output through max-threshold
            
            n_samples += outputs.shape[0]
            n_correct += (predictions == label).sum().item() / predictions.shape[1]

            acc = 100.0 * n_corrects / n_samples
            print(f"accuracy: {acc}")


In [55]:
epochs = 10
train_loop(train_dataloader=train_dataloader, epochs=epochs)
test_loop(test_dataloader=test_dataloader)

torch.float64


RuntimeError: expected scalar type Double but found Float